In [1]:
import pandas as pd
import numpy as np
import pprint
import re
from urlextract import URLExtract
from bs4 import BeautifulSoup

import json

import google
from googlesearch import search

#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1500)
pd.set_option('display.max_colwidth', -1)

In [2]:
fname='/Users/aeevr/Documents/CodeForSanJose/heartofthevalley/artwork-data/consolidated-artworks.json'

# Reading each nest of the json into DF one at a time and saving back to json

In [3]:
#read consolidated-artworks.json into dataframe

df = pd.read_json(fname)

In [4]:
#notice 'type' is copied for each record but only one line in original json
#This will be problematic later on when we save the dataframe back to json
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 0 to 269
Data columns (total 2 columns):
features    270 non-null object
type        270 non-null object
dtypes: object(2)
memory usage: 6.3+ KB


In [5]:
#features_df is the DF that holds 'features'
#'properties' is the key that holds in data we want to clean
features_df=pd.read_json((df['features']).to_json(orient='records'))
print(features_df.columns)
features_df.head()

Index(['addToMap', 'errors', 'geometry', 'id', 'properties', 'type'], dtype='object')


,addToMap,errors,geometry,id,properties,type
0,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.88887, 37.32718]}",10667523,"{'title': 'San José', 'artist': '<a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>', 'description': 'This mural was painted by <a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>, and produced in collaboration with <a href='https://www.exhibitiondistrict.com/' target='_blank'>Exhibition District</a>. It is located on the exterior of the PWC Building on Balbach St. <br/> Source: Exhibition District, https://www.exhibitiondistrict.com/', 'address': '488 Almaden Blvd, Ste 1800', 'city': 'San José', 'country': 'United States', 'postalCode': '95113', 'state': 'CA'}",Feature
1,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.97798, 37.30554]}",23925665,"{'title': 'Nothing Is Too Good for the Working Class', 'artist': 'Carlos Rodriguez', 'description': 'This mural was painted by artist <a href='https://www.carlosrodriguezart.com/' target='_blank'>Carlos Rodriguez</a>. The murals are located at the Las Palmas Supermarket. This mural was sponsored by The Health Trust. Source: Carlos Rodriguez, https://www.carlosrodriguezart.com/', 'address': '1080 Saratoga Ave #15', 'city': 'San José', 'country': 'United States', 'postalCode': '95129', 'state': 'CA'}",Feature
2,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.88582, 37.33022]}",31154388,"{'title': 'Guardian Tiger', 'artist': '<a href='http://mikeborja.tumblr.com/' target='_blank'>Mike Borja</a>', 'website': 'http://mikeborja.tumblr.com/', 'description': 'This mural is the middle in a series of three murals on the exterior wall of The Studio gym, facing the E San Salvador St. This mural is painted by Mike Borja. <br/> Source: Lakey, Cherri. “Find Beauty in the Streets of San Jose This Summer.” KQED, KQED, 5 July 2016, www.kqed.org/arts/11741855/find-beauty-in-the-streets-of-san-jose-this-summer.', 'address': '396 S 1st St.', 'city': 'San José', 'country': 'United States', 'postalCode': '95113', 'state': 'CA'}",Feature
3,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.89577, 37.35107]}",38393353,"{'title': 'Kokeshis de San Jo', 'artist': '<a href='http://samrodriguezart.com/' target='_blank'>Sam Rodriguez</a>', 'description': 'This artbox was painted by <a href='http://samrodriguezart.com/' target='_blank'>Sam Rodriguez</a>. It is located on the corner of 5th and Taylor by Fuji Towers. <br/> Source: Empire Seven Studios, Rasteroids, and Genevieve Santos. “Japantown Artwalk Map.“ Japantown Artwalk, 2017, http://jtownartwalk.com/wp-content/uploads/2016/06/jtown-artwalk-map-1.pdf ', 'address': '200 E Taylor St', 'city': 'San José', 'country': 'United States', 'postalCode': '95112', 'state': 'CA'}",Feature
4,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.84637, 37.34036]}",45973257,"{'title': 'A Monument to the Last Barn', 'artist': 'Gloria Bornstein', 'description': 'A native of San José, Emma Prusch bequeathed her 47 acre dairy farm to the City in 1962. The deed to the City specified that the City “will make said park a place of relaxation, recreation, and enjoyment for the people of the City of San José, and in such a manner as will, to extent that such can be reasonably done, give to said park a rural county character and atmosphere.” <br/><br/> Emma Prusch Farm Park includes heritage, high density, and rare fruit orchards, a Plant Science Center, two community gardens, pollinator and butterfly gardens, as well as acres of lush green lawns for picnics and playing. It also has one of the largest free standing wooden barns in California, home to 4H large animals including beef cattle, sheep, and swine. <br/><br/> In 2006, Gloria Bornstein was inspired by the story of Emma Prusch, San José's rich agricultural history and the continuing traditions. <br/><br/>Source: City of San José Public Art Program, http://www.sanjoseca.gov/Facilities/Facility/Details/Public-Art-A-Monument-to-the-Last-Barn-E-442', 'address': '647 S King Rd', 'city

In [6]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 0 to 269
Data columns (total 6 columns):
addToMap      1 non-null float64
errors        4 non-null object
geometry      270 non-null object
id            270 non-null int64
properties    270 non-null object
type          270 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 14.8+ KB


In [7]:
#covert features_df['properties'] into a json and open into its own df==>properties_df
#this has the keys we want to clean!
properties_df=pd.read_json((features_df['properties']).to_json(orient='records'))
print(properties_df.columns)
print(properties_df.info())
properties_df.head()

Index(['address', 'artist', 'city', 'country', 'crossStreet', 'description', 'image', 'postalCode', 'sourceOfInformation', 'sourceURL', 'state', 'title', 'website'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 0 to 269
Data columns (total 13 columns):
address                270 non-null object
artist                 270 non-null object
city                   270 non-null object
country                168 non-null object
crossStreet            26 non-null object
description            270 non-null object
image                  132 non-null object
postalCode             270 non-null int64
sourceOfInformation    102 non-null object
sourceURL              102 non-null object
state                  270 non-null object
title                  270 non-null object
website                33 non-null object
dtypes: int64(1), object(12)
memory usage: 29.5+ KB
None


,address,artist,city,country,crossStreet,description,image,postalCode,sourceOfInformation,sourceURL,state,title,website
0,"488 Almaden Blvd, Ste 1800",<a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>,San José,United States,NaN,"This mural was painted by <a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>, and produced in collaboration with <a href='https://www.exhibitiondistrict.com/' target='_blank'>Exhibition District</a>. It is located on the exterior of the PWC Building on Balbach St. <br/> Source: Exhibition District, https://www.exhibitiondistrict.com/",NaN,95113,NaN,NaN,CA,San José,NaN
1,1080 Saratoga Ave #15,Carlos Rodriguez,San José,United States,NaN,"This mural was painted by artist <a href='https://www.carlosrodriguezart.com/' target='_blank'>Carlos Rodriguez</a>. The murals are located at the Las Palmas Supermarket. This mural was sponsored by The Health Trust. Source: Carlos Rodriguez, https://www.carlosrodriguezart.com/",NaN,95129,NaN,NaN,CA,Nothing Is Too Good for the Working Class,NaN
2,396 S 1st St.,<a href='http://mikeborja.tumblr.com/' target='_blank'>Mike Borja</a>,San José,United States,NaN,"This mural is the middle in a series of three murals on the exterior wall of The Studio gym, facing the E San Salvador St. This mural is painted by Mike Borja. <br/> Source: Lakey, Cherri. “Find Beauty in the Streets of San Jose This Summer.” KQED, KQED, 5 July 2016, www.kqed.org/arts/11741855/find-beauty-in-the-streets-of-san-jose-this-summer.",NaN,95113,NaN,NaN,CA,Guardian Tiger,http://mikeborja.tumblr.com/
3,200 E Taylor St,<a href='http://samrodriguezart.com/' target='_blank'>Sam Rodriguez</a>,San José,United States,NaN,"This artbox was painted by <a href='http://samrodriguezart.com/' target='_blank'>Sam Rodriguez</a>. It is located on the corner of 5th and Taylor by Fuji Towers. <br/> Source: Empire Seven Studios, Rasteroids, and Genevieve Santos. “Japantown Artwalk Map.“ Japantown Artwalk, 2017, http://jtownartwalk.com/wp-content/uploads/2016/06/jtown-artwalk-map-1.pdf",NaN,95112,NaN,NaN,CA,Kokeshis de San Jo,NaN
4,647 S King Rd,Gloria Bornstein,San José,United States,NaN,"A native of San José, Emma Prusch bequeathed her 47 acre dairy farm to the City in 1962. The deed to the City specified that the City “will make said park a place of relaxation, recreation, and enjoyment for the people of the City of San José, and in such a manner as will, to extent that such can be reasonably done, give to said park a rural county character and atmosphere.” <br/><br/> Emma Prusch Farm Park includes heritage, high density, and rare fruit orchards, a Plant Science Center, two community gardens, pollinator and butterfly gardens, as well as acres of lush green lawns for picnics and playing. It also has one of the largest free standing wooden barns in California, home to 4H large animals including beef cattle, sheep, and swine. <br/><br/> In 2006, Gloria Bornstein was inspired by the story of Emma Prusch, San José's rich agricultural history and the continuing traditions. <br/><br/>Source: City of San José Public Art Program, http://www.sanjoseca.gov/Facilities/Facility/Details/Public-Art-A-Monument-to-the-Last-Barn-E-442",NaN,95116,NaN,NaN,CA,A Monument to the Last Barn,NaN


#  code to fix artist names and websites
* Extract simply name(s) from the artist property to put back into the artist property
* Extract URLs from from the artist property to merge into website property

In [8]:
#get new artist names
soup = BeautifulSoup(properties_df['artist'].iloc[0],'lxml')

tmp_col = properties_df['artist']
artist_name = []
artist_site = []
artist_leftover =[]

for i in range(len(tmp_col)):
    
    soup = BeautifulSoup(tmp_col.loc[i], 'lxml')
        
    artist_leftover.append(soup.get_text())

    tags = soup.find_all('a')
    
    if (len(tags)==1): 
        tags = soup.find('a')
        a_n = tags.text
        a_s = tags.attrs['href'];
        
            
    elif (len(tags)>1):
        multi_artist_dict = {}
        a_n = []
        a_s =[]
        for t in tags:
            a_n.append(t.text)
            a_s.append(t.attrs['href'])
        
    elif (len(tags)==0):
        a_n = tmp_col[i]
        a_s = ''
    
    artist_name.append(a_n)
    artist_site.append(a_s)
        
artist_df = pd.concat([properties_df['artist'],
                        pd.DataFrame(artist_leftover,columns=['artist_no_tags']),
                       pd.DataFrame(artist_name,columns=['artist_name']),
                       pd.DataFrame(artist_site,columns=['artist_site'])],axis=1)

KeyError: 'href'

# TYPO! in record 142 herf should be href
(causing artist name fixer to crash)

In [ ]:
properties_df['artist'].iloc[142]

In [9]:
properties_df['artist'].iat[142] = properties_df['artist'].iloc[142].replace('herf','href')
properties_df['artist'].iloc[142]

"<a href='https://deniskorkh.tumblr.com/' target='_blank'>Denis Korkh</a>"

# try to run artist name fixer code again

In [10]:
#get new artist names


#regex expression to split artists names esp. when multiple artists
artist_sep_pattern = re.compile(" and|\+|,|&|, |\u00B7")
#[^\u0000-\u007F][^\u0000-\u00FF]

tmp_col = properties_df['artist']
artist_name = []
artist_site = []
artist_leftover =[]

#loop over each entry in artist col
for i in range(len(tmp_col)):
    #parse entry in BS object
    soup = BeautifulSoup(tmp_col.loc[i], 'lxml')
    
    #the text without tags for QA purposes
    artist_leftover.append(soup.get_text())

    #after testing this is the text I want to use for artist_name
    a_n = soup.get_text()
    
    #If the artist_name has more than 4 'words' print because it is likely to be problematic
#     if (len(a_n.split())>5):
#         print('i=', i,' has more than 5 words so look closer, text = ',a_n,'\n')

    #check the artist name to see if more than one person, split the string and create a list
    if (len(re.split(artist_sep_pattern, a_n)) >1): 
        a_n = [i for i in re.split(artist_sep_pattern, a_n) if len(i)>0 ];

    #gets html tags with 'a' which is all is in this column
    tags = soup.find_all('a')

    
    #only 1 tag in entry, send the tag's attr to the artist_site column
    if (len(tags)==1): 
        tags = soup.find('a')
        a_s = tags.attrs['href'];
#         if ((properties_df['website'].isnull().iloc[i]==False) and 
#             (properties_df['website'].iloc[i]!=tags.attrs['href'])):             
#             a_s=[properties_df['website'].iloc[i],tags.attrs['href']];
#         else:             
#             a_s = tags.attrs['href'];
        #a_n = tags.text
        #check the artist name to see if more than one person and create a list
        #if (len(re.split(artist_sep_pattern, a_n)) >1): a_n = [i for i in re.split(artist_sep_pattern, a_n)];
        
    #more than one tag in entry, 
    #create list of websites connected to each tag & send to the artist_site column
    #print out info on these to edit manually if neccessary
    elif (len(tags)>1):
        multi_artist_dict = {}
        #a_n = []
        a_s =[]
        for t in tags:
            #a_n.append(t.text)
            a_s.append(t.attrs['href'])
#             print('i=', i,' has more than 1 tag so look closer, tag = ',t)
#             print(t.text, t.attrs['href'])
#         print('\n')
    
    #if no tags (desired behavior!) the the text should b the artist name
    elif (len(tags)==0):
        a_n = tmp_col[i]
        a_s=np.nan
    
    artist_name.append(a_n)
    artist_site.append(a_s)
        
artist_df = pd.concat([properties_df['artist'],
                       properties_df['website'],
                       pd.DataFrame(artist_site,columns=['artist_site']),
                       pd.DataFrame(artist_leftover,columns=['artist_no_tags']),
                       pd.DataFrame(artist_name,columns=['artist_name'])],axis=1)


artist_df.head()

,artist,website,artist_site,artist_no_tags,artist_name
0,<a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>,NaN,https://www.instagram.com/j.duh_/,J.Duh,J.Duh
1,Carlos Rodriguez,NaN,NaN,Carlos Rodriguez,Carlos Rodriguez
2,<a href='http://mikeborja.tumblr.com/' target='_blank'>Mike Borja</a>,http://mikeborja.tumblr.com/,http://mikeborja.tumblr.com/,Mike Borja,Mike Borja
3,<a href='http://samrodriguezart.com/' target='_blank'>Sam Rodriguez</a>,NaN,http://samrodriguezart.com/,Sam Rodriguez,Sam Rodriguez
4,Gloria Bornstein,NaN,NaN,Gloria Bornstein,Gloria Bornstein


# Merge ['artist_site'] into ['website']

In [11]:
#This shows that the non-null entries in artist_df['website'] are identical to artist_df['artist_site']
#EXCEPT entry 105
print(artist_df[artist_df['website'].isnull()==False]['website']==artist_df[artist_df['website'].isnull()==False]['artist_site'])
print('\n')

#This shows that artist_df['artist_site'] pulled more info than was contained in artist_df['website']:
#website           33 non-null object
#artist_site       90 non-null object

#Therefore, it is reasonable to replace artist_df['website'] with artist_df['artist_site']
#EXCEPT entry 105
print(artist_df.info())

artist_df[['website','artist_site']].iloc[99:110]

#replace null vales of 'website' with non-null values of 'artist_site
artist_df.loc[((artist_df['artist_site'].isnull()==False) & 
                   (artist_df['website'].isnull()==True)),
                  'website'] = artist_df['artist_site']

print(artist_df[artist_df['website'].isnull()==False]['website']==artist_df[artist_df['website'].isnull()==False]['artist_site'])
print('\n')
print(artist_df.info())
#website           33 -> 91 non-null object
#artist_site       90 ->90 non-null object [UNCHANGED]


artist_df[['website','artist_site']].iloc[99:110]

2      True 
7      True 
9      True 
12     True 
20     True 
22     True 
41     True 
69     True 
77     True 
78     True 
89     True 
94     True 
96     True 
97     True 
105    False
107    True 
109    True 
114    True 
119    True 
142    True 
148    True 
159    True 
163    True 
187    True 
195    True 
205    True 
208    True 
209    True 
219    True 
225    True 
230    True 
241    True 
265    True 
dtype: bool


<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 0 to 269
Data columns (total 5 columns):
artist            270 non-null object
website           33 non-null object
artist_site       90 non-null object
artist_no_tags    270 non-null object
artist_name       270 non-null object
dtypes: object(5)
memory usage: 22.7+ KB
None
0      True 
2      True 
3      True 
7      True 
8      True 
9      True 
10     True 
11     True 
12     True 
18     True 
20     True 
22     True 
29     True 
37     True 
39     True 
41     True 
42     True

,website,artist_site
99,NaN,NaN
100,NaN,NaN
101,https://www.instagram.com/travmsk/,https://www.instagram.com/travmsk/
102,https://www.instagram.com/nandoid2485/,https://www.instagram.com/nandoid2485/
103,NaN,NaN
104,NaN,NaN
105,http://www.andrewschoultz.com/,NaN
106,http://www.sm-artgroup.com/,http://www.sm-artgroup.com/
107,http://samrodriguezart.com/,http://samrodriguezart.com/
108,"[http://instagram.com/gigamelted/, http://instagram.com/chgyyrscrenammme/, http://instagram.com/griffinone/, https://www.instagram.com/wispertfl/]","[http://instagram.com/gigamelted/, http://instagram.com/chgyyrscrenammme/, http://instagram.com/griffinone/, https://www.instagram.com/wispertfl/]"


# Fix entry 160 which has all the description text stuck in artist_name

In [12]:
artist_df['artist_name'].iat[160]

'Gerald Heffernon This fun and exuberant\xa0sculpture links the themes of wild animal and domestic dog. A bronze beaver in the creek slaps the water with its tail and creates an over-sized splash of stainless steel.\xa0 An extension of the splash sends a bright yellow ball in a high-arcing trajectory ten feet high.\xa0  Fifteen feet away, a bronze dog leaps ten feet in the air and in visual time lapse sequence catches the ball, completing the connection between the two sculptural elements and the two worlds.\xa0\xa0District6\xa0Report a concern\xa0(408) 793-04330 or\xa0publicart@sanjoseca.go'

In [13]:
artist_df['artist_name'].iat[160] = 'Gerald Heffernon'

In [14]:
artist_df['artist_name'].iloc[160]

'Gerald Heffernon'

In [15]:
#check 'description' of this entry because maybe this text needs to be moved there
#Clearly, not an issue
properties_df['description'].iloc[160]

'This fun and exuberant\xa0sculpture links the themes of wild animal and domestic dog. A bronze beaver in the creek slaps the water with its tail and creates an over-sized splash of stainless steel.\xa0 An extension of the splash sends a bright yellow ball in a high-arcing trajectory ten feet high.\xa0  Fifteen feet away, a bronze dog leaps ten feet in the air and in visual time lapse sequence catches the ball, completing the connection between the two sculptural elements and the two worlds.\xa0\xa0District6\xa0Report a concern\xa0(408) 793-04330 or\xa0publicart@sanjoseca.gov'

# Code to make a Dataframe with google results for the artist.

## Save to a csv, review/edit by hand, reincorporate results




### Markdown to prevent re-running!

```python
artist_gsearch_string = artist_df.artist_no_tags
artist_gsearch_string.iloc[160] = 'Gerald Heffernon'

artistURL = []

for at in artist_gsearch_string:
    srch_str=at+' artist San Jose'
    gr_list=[]
    [gr_list.append(gr) for gr in search(srch_str, tld='com', lang='en', num=2, start=0, stop=1, pause=2.0)]
    print(at,gr_list)
    artistURL.append(gr_list)
    
artist_df['artistGGL'] = artistURL
artist_df[['artist_no_tags','artistGGL','website']].sort_values('artist_no_tags').to_csv('artist.csv',sep='\t')
```

# Replace [website] with ['artistGGL'] from artist_hand_edit.csv AND

## Remove the following entries:
- 83 (copy of 17)
- 179 (copy of 117)
- 59 (copy of 130)
- 173 (copy of 134)
- 14 [mural gone]
- 82,184 (both copy of 43)
- 137 (copy of 27)
- 227 (copy of 61)
- 264 (copy of 171)
- 106,178 (copy of 46)


## Fix the following names:
- 0 'Jorge Camacho'
- 108 ['Amine Rastgar', 'Robbie Api', 'Griffin, Doper', 'Tomas Wisper Talamantes']
- 161 Andrew Leicester (misspell)
- 154 Ben Henderson (remove other names from good karma brewery mural)
- 75 ['Mark Evans', 'Charley Brown'] (edit names)
- 163 Fernando “FORCE129” Amaro Jr' (edit name) 
- 28 Living Lenses (Po Shu Wang and Louise Bertelsen) (fix names)
- 207,250,198,39,248,136 MMAP
- 132 Mr. and Mrs. Fergeson (misspell)
- 119 Yis 'Nosego' Goodwin
- 84 Jeff Gress (not pro_gress)
- 151 San Jose State Art in the Community Class
- 47,196 ['Tony May (Instructor)','San Jose State Art in the Community Class']
- 242 Serina Serjama
- 216 ['Liz Utson','Stephen DeStabler','Tom Aidala','Manuel Neri','Nathan Oliviera','Sam Hernandez','John Buck','A.D.M. Cooper', 'Julia Lord']
- 223 ['Benjamino Bufano','Brian Holt', 'Kenneth Matsumoto']
- 224 ['Barbara Kasten','Ed Moses','Selina Trieff','Rupert Garcia','Katherine Levin-Lau']
- 237  ['Mike Bondi', 'Alan Masaoka',' Tom Stanton',' Penelope Fried',' Pam Morris',' Rick Arnitz',' Brian Blood', ' Jamie Brunson',' Willard Dixon','Wade Hoefer',' Amy Kaufman',' Jeff Long', 'Carloe Pierce',' Silvia Poloto']
- 177 ['Various Artists','Tomas Wisper Talamantes']


## Fix the following titles:
- 117 title='Pacha Mama'


In [16]:
ggl_df = pd.read_csv('artist_hand_edit.csv')

In [17]:
#add the ['artistGGL'] to artist_df
artist_df['artistGGL'] = ggl_df.artistGGL
#print to check visually
artist_df[['artist_name','artistGGL','website','artist_site']].to_html('/tmp/artist_df_table.html')

In [18]:
#propagating artist names, website back to properties_df

properties_df['artist'] = artist_df.artist_name
properties_df['website'] = artist_df.artistGGL


In [19]:
#0 'Jorge Camacho'
print(properties_df.iloc[0].artist)
properties_df.artist.iat[0] = 'Jorge Camacho'
print(properties_df.iloc[0].artist)

J.Duh
Jorge Camacho


In [20]:
#108 ['Amine Rastgar', 'Robbie Api', 'Griffin, Doper', 'Tomas Wisper Talamantes']
print(properties_df.iloc[108].artist)
properties_df.artist.iat[108] = ['Amine Rastgar', 'Robbie Api', 'Griffin, Doper', 'Tomas Wisper Talamantes']
print(properties_df.iloc[108].artist)


['Amine Rastgar', ' Robbie Api', ' Griffin', ' Doper', ' Wisper.  Source: POW! WOW! San José', ' 2017 Mural ', ' Event Locations https://www.powwowsanjose.com/2017-locations/']
['Amine Rastgar', 'Robbie Api', 'Griffin, Doper', 'Tomas Wisper Talamantes']


In [21]:
# 161 Andrew Leicester (misspell)
print(properties_df.iloc[161].artist)
properties_df.artist.iat[161] = 'Andrew Leicester'
print(properties_df.iloc[161].artist)

Andrew Leister
Andrew Leicester


In [22]:
# 154 Ben Henderson (remove other names from good karma brewery mural)
print(properties_df.iloc[154].artist)
properties_df.artist.iat[154] = 'Ben Henderson'
print(properties_df.iloc[154].artist)

['Ben Henderson', ' Lacey Bryant', ' Drew Roulette', ' Sir Beardo']
Ben Henderson


In [23]:
# 75 ['Mark Evans', 'Charley Brown'] (edit names)
print(properties_df.iloc[75].artist)
properties_df.artist.iat[75] = ['Mark Evans', 'Charley Brown']
print(properties_df.iloc[75].artist)

Evans & Brown
['Mark Evans', 'Charley Brown']


In [24]:
# 163 Fernando “FORCE129” Amaro Jr' (edit name) 
print(properties_df.iloc[163].artist)
properties_df.artist.iat[163] = 'Fernando “FORCE129” Amaro Jr'
print(properties_df.iloc[163].artist)

Force129
Fernando “FORCE129” Amaro Jr


In [25]:
# 28 Living Lenses (Po Shu Wang and Louise Bertelsen) (fix names)
print(properties_df.iloc[28].artist)
properties_df.artist.iat[28] = ['Po Shu Wang','Louise Bertelsen']
print(properties_df.iloc[28].artist)

Living Lenses – Po Shu Wang and Louise Bertelsen
['Po Shu Wang', 'Louise Bertelsen']


In [26]:
# 207,250,198,39,248,136 MMAP
print(properties_df.iloc[[207,250,198,39,248,136]].artist)
properties_df.artist.iat[207] = 'Scape Martinez'
properties_df.artist.iat[250] = ['Sarah Woodward','Scape Martinez','Laura Nagle','Yetunde Olagbaju','Randy Reynolds','NVIDIA volunteers']
properties_df.artist.iat[198] = ['Sarah Woodward', 'students, staff, parents of César Chávez Academy and Ravenswood City School District']
properties_df.artist.iat[39] = ['Omar Ramirez','students and staff at Boys & Girls Clubs of the Peninsula, Moldaw-Zaffaroni Clubhouse']
properties_df.artist.iat[248] = ['Omar Ramirez','students, staff and community members at the East Palo Alto Charter School']
properties_df.artist.iat[136] = ['Omar Ramirez','students, staff, and volunteers of the Teen Mural Project']
print(properties_df.iloc[[207,250,198,39,248,136]].artist)


207    [MMAP (Mural Music,  Arts Project),  Scape Martinez]                                                                                                     
250    [MMAP (Mural Music,  Arts Project) team,  including Sarah Woodward,  Scape Martinez,  Laura Nagle,  Yetunde Olagbaju; Randy Reynolds,  NVIDIA volunteers]
198    [MMAP (Mural Music,  Arts Project),  lead artist Sarah Woodward,  students,  staff,  parents of César Chávez Academy,  Ravenswood City School District]  
39     [MMAP (Mural Music,  Arts Project),  lead artist: Omar Ramirez,  students,  staff]                                                                       
248    [MMAP (Mural Music,  Arts Project),  lead artist: Omar Ramirez,  students,  staff,  community members]                                                   
136    [MMAP (Mural Music,  Arts Project),  lead artist: Omar Ramirez,  students,  staff,  volunteers of the Teen Mural Project]                                
Name: artist, dtype: object
207   

In [27]:
# 132 Mr. and Mrs. Fergeson (misspell)
print(properties_df.iloc[132].artist)
properties_df.artist.iat[132] = 'Mr. and Mrs. Ferguson'
print(properties_df.iloc[132].artist)

Mr. and Mrs. Fergeson
Mr. and Mrs. Ferguson


In [28]:
# 119 Yis 'Nosego' Goodwin
print(properties_df.iloc[119].artist)
properties_df.artist.iat[119] = 'Yis "NoseGo" Goodwin'
print(properties_df.iloc[119].artist)


NoseGo
Yis "NoseGo" Goodwin


In [29]:
# 84 Jeff Gress (not pro_gress)
print(properties_df.iloc[84].artist)
properties_df.artist.iat[84] = 'Jeff Gress'
print(properties_df.iloc[84].artist)


pro_gress
Jeff Gress


In [30]:
# 151 SJSU Art in the Community Class
print(properties_df.iloc[151].artist)
properties_df.artist.iat[151] = 'SJSU Art in the Community Class'
print(properties_df.iloc[151].artist)


San Jose State Art & Community Class
SJSU Art in the Community Class


In [31]:
# 47,196 ['Tony May (Instructor)','San Jose State Art in the Community Class']
print(properties_df.iloc[[47,196]].artist)
properties_df.artist.iat[47] = ['Tony May (Instructor)','SJSU Art in the Community Class']
properties_df.artist.iat[196] = ['Tony May (Instructor)','SJSU Art in the Community Class']
print(properties_df.iloc[[47,196]].artist)


47     San Jose State University “Art in Community” Students with Tony May, Instructor
196    Tony May with Art in Community Class - SJSU                                    
Name: artist, dtype: object
47     [Tony May (Instructor), SJSU Art in the Community Class]
196    [Tony May (Instructor), SJSU Art in the Community Class]
Name: artist, dtype: object


In [32]:
# 242 Serina Serjama
print(properties_df.iloc[242].artist)
properties_df.artist.iat[242] = 'Serina Serjama'
print(properties_df.iloc[242].artist)

['Serina Serjama', ' in collaboration with Women Rising San José', ' Heart of Chaos']
Serina Serjama


In [33]:
# 216 ['Liz Utson','Stephen DeStabler','Tom Aidala','Manuel Neri','Nathan Oliviera','Sam Hernandez','John Buck','A.D.M. Cooper', 'Julia Lord']
print(properties_df.iloc[216].artist)
properties_df.artist.iat[216] = ['Liz Utson','Stephen DeStabler','Tom Aidala','Manuel Neri',
                                 'Nathan Oliviera','Sam Hernandez','John Buck','A.D.M. Cooper',
                                 'Julia Lord']
print(properties_df.iloc[216].artist)


Various
['Liz Utson', 'Stephen DeStabler', 'Tom Aidala', 'Manuel Neri', 'Nathan Oliviera', 'Sam Hernandez', 'John Buck', 'A.D.M. Cooper', 'Julia Lord']


In [34]:
# 223 ['Benjamino Bufano','Brian Holt', 'Kenneth Matsumoto']
print(properties_df.iloc[223].artist)
properties_df.artist.iat[223] = ['Benjamino Bufano','Brian Holt','Kenneth Matsumoto']
print(properties_df.iloc[223].artist)


Various
['Benjamino Bufano', 'Brian Holt', 'Kenneth Matsumoto']


In [35]:
# 224 ['Barbara Kasten','Ed Moses','Selina Trieff','Rupert Garcia','Katherine Levin-Lau']
print(properties_df.iloc[224].artist)
properties_df.artist.iat[224] = ['Barbara Kasten','Ed Moses','Selina Trieff','Rupert Garcia','Katherine Levin-Lau']
print(properties_df.iloc[224].artist)


Various
['Barbara Kasten', 'Ed Moses', 'Selina Trieff', 'Rupert Garcia', 'Katherine Levin-Lau']


In [36]:
# 237  ['Mike Bondi', 'Alan Masaoka',' Tom Stanton',' Penelope Fried',' Pam Morris',' Rick Arnitz',' Brian Blood', ' Jamie Brunson',' Willard Dixon','Wade Hoefer',' Amy Kaufman',' Jeff Long', 'Carloe Pierce',' Silvia Poloto']
print(properties_df.iloc[237].artist)
properties_df.artist.iat[237] = ['Mike Bondi','Alan Masaoka','Tom Stanton','Penelope Fried',
                                 'Pam Morris','Rick Arnitz','Brian Blood', 'Jamie Brunson',
                                 'Willard Dixon','Wade Hoefer','Amy Kaufman','Jeff Long',
                                 'Carloe Pierce','Silvia Poloto']
print(properties_df.iloc[237].artist)

Various
['Mike Bondi', 'Alan Masaoka', 'Tom Stanton', 'Penelope Fried', 'Pam Morris', 'Rick Arnitz', 'Brian Blood', 'Jamie Brunson', 'Willard Dixon', 'Wade Hoefer', 'Amy Kaufman', 'Jeff Long', 'Carloe Pierce', 'Silvia Poloto']


In [37]:
# 177 ['Various Artists','Tomas Wisper Talamantes']
print(properties_df.iloc[177].artist)
properties_df.artist.iat[177] = ['Various Artists','Tomas Wisper Talamantes']
print(properties_df.iloc[177].artist)

Various Artists
['Various Artists', 'Tomas Wisper Talamantes']


In [38]:
#117 title='Pacha Mama'
print(properties_df.iloc[117].title)
properties_df.title.iat[117] = 'Pacha Mama'
print(properties_df.iloc[117].title)

Señora de Los Nopales
Pacha Mama


In [39]:
#dropping rows from properties_df
print(properties_df.shape)
properties_df = properties_df.drop([83,179,59,173,14,82,184,137,227, 264, 106,178])
print(properties_df.shape)

properties_df.reset_index(drop=True,inplace=True)
properties_df.columns

(270, 13)
(258, 13)


Index(['address', 'artist', 'city', 'country', 'crossStreet', 'description', 'image', 'postalCode', 'sourceOfInformation', 'sourceURL', 'state', 'title', 'website'], dtype='object')

# Moving on to fix description column. 

In [40]:
#get new desc names

tmp_col = properties_df['description']
desc_name = []
desc_site = []
desc_leftover =[]
desc_source=[]
source_site=[]

extractor = URLExtract()
extractor.update()

the_entry = -1000; # for printing info about a single entry 4 debug

#loop over each entry in desc col
for i in range(len(tmp_col)):
    #if (i!=the_entry): continue;
    ref=[]
    
    #parse entry in BS object
    soup = BeautifulSoup(tmp_col.loc[i], 'lxml')

    #after testing this is the text I want to use for desc_name
    a_n = soup.get_text()
    if (i==the_entry): print('1: description=soup.get_text():',a_n)
    #remove weird \r\n that is in some entries
    a_n = re.sub('\\r\\n','',a_n)
    if (i==the_entry): print('2:description=soup.get_text():',a_n)

    #for i=68
    if (a_n=='Text'): a_n='';
    
    source = ''
    url=[]
    #extract text starting with 'Source'
    if (a_n.find('Source')!=-1):
        
        #find index in string where the S in Source is
        source=a_n[a_n.find('Source'):]
        if (i==the_entry):print('source1:',source)

        #remove source string from descrption
        #a_n = re.sub('Source.*:','',a_n)
        a_n = re.sub(source,'',a_n)
        if (i==the_entry):print('source txt shoulda been stripped frm DESC here:',a_n)

            
        #removes the Source and whatever comes after up to the :    
        source=re.sub('^Source.*: ','',source) 
        #for entry 94
        #source= re.sub('Source - ','',source)
        if (i==the_entry):print('source2 (supposed to remove "Source*:"):',source)    

        #make sure only one source site
        if (len(source)<1): source='';    
        elif (source[len(source)-1]=='.'): source = source[:len(source)-1];
        if (i==the_entry):print('source3 (supposed to remove period @ end):',source)    
    
        #remove source string from descrption
        #a_n = re.sub('Source.*:','',a_n)
            
        #extract url from source text
        url = extractor.find_urls(source)
        if (i==the_entry): 
            print('url in source str:',url)
            print('url[0]: ', url[0])
            print('len(url):',len(url))
            print('type(url[0]):',type(url[0]))

        #no source text has more than one link
        if (len(url)>1): print('source has more than one link i:',i, len(url));

        #remove url from source leaving name hopefully
        if (i==the_entry):print('source4:',source)    
        if (len(url)>0): source=re.sub(re.escape(url[0]),'',source);
        if (i==the_entry):
            print('source5:',source)
            tmp=str(source)
            print('re.sub(url[0],'',source)',re.sub('http','',tmp))

        #remove ,/ from end of source
        source=re.sub('\W+\Z','',source)
        if (i==the_entry):print('source with url removed:',source)
    desc_source.append(source)
    source_site.append(url)

    #if (i==the_entry): print('source_site:',source_site)
    
    #gets html tags with 'a' which is all is in this column
    tags = soup.find_all('a')
    if (i==the_entry):print('tags:',tags)
    
    #only 1 tag in entry, send the tag's attr to the desc_site column
    if (len(tags)==1): 
        tags = soup.find('a')
        single_desc_ref = tags.attrs['href']
        ref.append(tags.attrs['href'])
        
    #more than one tag in entry, 
    #create list of websites connected to each tag & send to the desc_site column
    #print out info on these to edit manually if neccessary
    elif (len(tags)>1):
        #remove duplicates in ref list
        ref_list=list(set([t.attrs['href'] for t in tags]))
        [ref.append(x) for x in ref_list]
        #remove duplicates in ref list
        
    if (i==the_entry):print('ref:',ref)
    
    desc_name.append(a_n)
    
    
    
    
    
    #check if link is in properties['website'] or source_site [=URLs stripped from 'description']
    #and don't save if it is
    #if ref is empty list, save it and move on
    if ref:
        for r in ref:
            if ((properties_df['website'].isnull().iloc[i]==False) and
                (not(isinstance(properties_df['website'].iloc[i],list))) and
                ((r==properties_df['website'].iloc[i]) or 
                (r[:-1]==properties_df['website'].iloc[i]) or
                (r==properties_df['website'].iloc[i][:-1]))):
                ref.remove(r)
    
    if ref:
        for r in ref:
            if (isinstance(properties_df['website'].iloc[i],list)):
                for aws in properties_df['website'].iloc[i]:
                    if ((r==aws) or (r[:-1]==aws) or (r==aws[:-1])):
                        #print('REMOVED\n\n')
                        ref.remove(r)
            
    if ref:
        for r in ref:                               
            if ((source_site[i]!=np.nan) and
                (not(isinstance(source_site[i],list))) and
                ((r==source_site[i]) or 
                (r[:-1]==source_site[i]) or
                (r==source_site[i][:-1]))):
                print("GOT HERE!!!!")
                ref.remove(r)

    if ref:
        for r in ref:                
            if (isinstance(source_site[i],list)):
                for ss in source_site[i]:
                    if ((r==ss) or (r[:-1]==ss) or (r==ss[:-1])):
                        #print('\n',isinstance(source_site[i],list),source_site[i], ref, r)
                        ref.remove(r)           
                        
    desc_site.append([ref])    
    

desc_df = pd.concat([properties_df['description'],
                     pd.DataFrame(desc_name,columns=['desc_name']),
                     pd.DataFrame(desc_source,columns=['desc_source']),
                     pd.DataFrame(source_site,columns=['source_site']),
                     pd.DataFrame(desc_site,columns=['desc_site']),
                     properties_df['website'],
                     properties_df['sourceURL'],
                     properties_df['sourceOfInformation']], 
                     axis=1)


if (the_entry >= 0):
    print('\n',the_entry,desc_df.iloc[the_entry],'\n')
    desc_df.iloc[the_entry-1:the_entry+2]

desc_df.tail(25)


,description,desc_name,desc_source,source_site,desc_site,website,sourceURL,sourceOfInformation
233,"Cyclorilla was painted by <a href='https://www.instagram.com/seanboyles38th/' target='_blank'>Sean Boyles</a>. This mural was exhibited at the 2017 <a href='https://www.powwowsanjose.com/' target='_blank'>POW! WOW! San José</a> in collaboration with <a href='https://www.empiresevenstudios.com' target='_blank'>Empire Seven Studios</a>. <br/> Source: Empire Seven Studios, Rasteroids, and Genevieve Santos. “Japantown Artwalk Map.“ Japantown Artwalk, 2017, http://jtownartwalk.com/wp-content/uploads/2016/06/jtown-artwalk-map-1.pdf",Cyclorilla was painted by Sean Boyles. This mural was exhibited at the 2017 POW! WOW! San José in collaboration with Empire Seven Studios.,"Empire Seven Studios, Rasteroids, and Genevieve Santos. “Japantown Artwalk Map.“ Japantown Artwalk, 2017",http://jtownartwalk.com/wp-content/uploads/2016/06/jtown-artwalk-map-1.pdf,"[https://www.empiresevenstudios.com, https://www.powwowsanjose.com/]",https://www.instagram.com/seanboyles38th/,NaN,NaN
234,"Waterscape\r\nArtists: Anna Valentina Murch and Doug Hollis\r\n\r\nThis water sculpture was inspired by the natural geological and artesian conditions underlying the San Jose region. \r\n\r\nThe specific design of Waterscape is composed of two topographic fields of monumental granite slabs. The fields slope from Santa Clara Street toward the City Hall plaza allowing the water to flow down the surface of the fields onto the plaza in subtle ripples. Water also flows over the vertical stone on the fountain edges, disappearing beneath the plaza, creating walls of water that define the perimeter along Santa Clara Street. The water within the field is programmed to respond to changes in time of day, temperature, season and availability of water to create different visual, sonic and atmospheric qualities. The piece is illuminated at night for a distinctive evening identity.","WaterscapeArtists: Anna Valentina Murch and Doug HollisThis water sculpture was inspired by the natural geological and artesian conditions underlying the San Jose region. The specific design of Waterscape is composed of two topographic fields of monumental granite slabs. The fields slope from Santa Clara Street toward the City Hall plaza allowing the water to flow down the surface of the fields onto the plaza in subtle ripples. Water also flows over the vertical stone on the fountain edges, disappearing beneath the plaza, creating walls of water that define the perimeter along Santa Clara Street. The water within the field is programmed to respond to changes in time of day, temperature, season and availability of water to create different visual, sonic and atmospheric qualities. The piece is illuminated at night for a distinctive evening identity.",,None,[],"['https://en.wikipedia.org/wiki/Anna_Valentina_Murch','http://www.doxastic.com/douglas/']",http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Waterscape-San-Jose-City-Hall-364,http://sanjoseca.gov/Facilities
235,"Plumed Serpent is a stylized image of Quetzalcóatl, one of the most important mythological deities of the Mesoamerican pantheon. It was created for the City of San Jose to symbolize the spirit of social harmony and diversity.","Plumed Serpent is a stylized image of Quetzalcóatl, one of the most important mythological deities of the Mesoamerican pantheon. It was created for the City of San Jose to symbolize the spirit of social harmony and diversity.",,None,[],['http://www.robertgraham-artist.com/'],http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Plumed-Serpent-Plaza-de-Csar--498,http://sanjoseca.gov/Facilities
236,"This public artwork is located at Alum Rock Youth Center. In 2003, Ries Niemi collaborated with nearby middle and high school students to incorporate their drawings as metal panels decorating the opening gate, front doorway, and walls of the youth center. While the artist translated some of the drawings for a

In [41]:
#'sourceOfInformation' seems silly since it is the beginning part of the URL in sourceURL
#[ALWAYS 'sourceOfInformation'=http://sanjoseca.gov/Facilities]
#SO I will change it to be my desc_df['desc_source'] which is the text name of the source
#IOW set desc_df['sourceOfInformation'] to be desc_df['desc_source']

desc_df[desc_df['sourceOfInformation'].isnull()==False][['sourceURL','sourceOfInformation']].head()

,sourceURL,sourceOfInformation
5,http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-CONNECTED-Silicon-Valley-Bang-409,http://sanjoseca.gov/Facilities
6,http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Convergence-San-Jose-Repertor-418,http://sanjoseca.gov/Facilities
13,http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Ol-Rose-Garden-Branch-Library-475,http://sanjoseca.gov/Facilities
18,http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Wind-in-the-Willows-Willow-Gl-483,http://sanjoseca.gov/Facilities
20,http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Frontier-Village-Birdhouses-E-451,http://sanjoseca.gov/Facilities


In [42]:
desc_df['sourceOfInformation'] = desc_df['desc_source']
desc_df[['sourceURL','sourceOfInformation']].tail(20)

,sourceURL,sourceOfInformation
238,http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Capiz-JTS-Northside-Community-457,
239,NaN,MMAP
240,http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Sensing-YOU-Sensing-WATER-Hwy-555,
241,NaN,City of San José Public Art Program
242,NaN,Paul J. Gonzalez Art Studios
243,NaN,Paul J. Gonzalez Art Studios
244,NaN,City of San José Public Art Program
245,http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Remembering-Agriculture-521,
246,NaN,Paul J. Gonzalez Art Studios
247,http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Spirit-Gate-Hester-Elementary-444,


In [43]:
#Can replace null values of sourceURL with non-null values of source_site because they are 
#never populated at the same time in a single entry 

desc_df[ (desc_df['sourceURL'].isnull()==False) &  
       (desc_df['source_site'].isnull()==False)][['source_site','sourceURL']]

,source_site,sourceURL


In [44]:
desc_df[ (desc_df['sourceURL'].isnull()==True) & (desc_df['source_site'].isnull()==False)][['sourceURL','source_site']]

,sourceURL,source_site
0,NaN,https://www.exhibitiondistrict.com
1,NaN,https://www.carlosrodriguezart.com
2,NaN,www.kqed.org/arts/11741855/find-beauty-in-the-streets-of-san-jose-this-summer
3,NaN,http://jtownartwalk.com/wp-content/uploads/2016/06/jtown-artwalk-map-1.pdf
4,NaN,http://www.sanjoseca.gov/Facilities/Facility/Details/Public-Art-A-Monument-to-the-Last-Barn-E-442
7,NaN,www.kqed.org/arts/11741855/find-beauty-in-the-streets-of-san-jose-this-summer
8,NaN,http://www.gemellosmurals.com/portfolio/life-in-willow-glen/
10,NaN,www.kqed.org/arts/11741855/find-beauty-in-the-streets-of-san-jose-this-summer
11,NaN,https://www.instagram.com/fco1980/.
12,NaN,https://www.instagram.com/fco1980/


In [45]:
#replace null values of sourceURL with non-null values of source_site
desc_df.loc[((desc_df['source_site'].isnull()==False) & 
             (desc_df['sourceURL'].isnull()==True)),
                'sourceURL'] = desc_df['source_site']
desc_df[['sourceURL','source_site']]

,sourceURL,source_site
0,https://www.exhibitiondistrict.com,https://www.exhibitiondistrict.com
1,https://www.carlosrodriguezart.com,https://www.carlosrodriguezart.com
2,www.kqed.org/arts/11741855/find-beauty-in-the-streets-of-san-jose-this-summer,www.kqed.org/arts/11741855/find-beauty-in-the-streets-of-san-jose-this-summer
3,http://jtownartwalk.com/wp-content/uploads/2016/06/jtown-artwalk-map-1.pdf,http://jtownartwalk.com/wp-content/uploads/2016/06/jtown-artwalk-map-1.pdf
4,http://www.sanjoseca.gov/Facilities/Facility/Details/Public-Art-A-Monument-to-the-Last-Barn-E-442,http://www.sanjoseca.gov/Facilities/Facility/Details/Public-Art-A-Monument-to-the-Last-Barn-E-442
5,http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-CONNECTED-Silicon-Valley-Bang-409,None
6,http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Convergence-San-Jose-Repertor-418,None
7,www.kqed.org/arts/11741855/find-beauty-in-the-streets-of-san-jose-this-summer,www.kqed.org/arts/11741855/find-beauty-in-the-streets-of-san-jose-this-summer
8,http://www.gemellosmurals.com/portfolio/life-in-willow-glen/,http://www.gemellosmurals.com/portfolio/life-in-willow-glen/
9,NaN,None


# Combine results back into the original 'Properties' dataframe

1. properties_df['description'] replaced with desc_df['desc_name']
1. properties_df['sourceOfInformation'] should be replaced with desc_df['sourceOfInformation'] 
1. properties_df['sourceURL'] should be replaced with desc_df['sourceURL'] 
1. Create NEW  property: properties_df['AdditionalURL'] from desc_df['desc_site']

In [46]:
properties_df['description']         = desc_df['desc_name']
properties_df['sourceOfInformation'] = desc_df['sourceOfInformation']
properties_df['sourceURL']           = desc_df['sourceURL']
properties_df['AdditionalURL']       = desc_df['desc_site']

In [47]:
#Fix 251 where the description had a sentence beginning with the word 'Source' 
print(properties_df.sourceOfInformation.iloc[251])
properties_df.sourceOfInformation.iat[251] = []
print(properties_df.sourceOfInformation.iloc[251])


print(properties_df.description.iloc[251])
properties_df.description.iat[251] = 'Internationally-recognized photographer Catherine Wagner created a dynamic architectural frieze that investigates ideas about knowledge, nature, wonder and discovery. This frieze is installed on the clerestory walls of the library entrance, and consists of printed images laminated to aluminum panels. Source material for the photos includes trees, rocks and plants from Overfelt Gardens, as well as Chinese Scholars’ Rocks from some of the preeminent collections in the United States.Integrated into the installation at the west end of the building is the signage for the Library’s “Tech Connect” area. This sign is comprised of two parts—the practical wayfinding text, and the surrounding cellular image forms. The cellular pattern was made using Voronoi diagrams for points that are algorithmically distributed through the panel and concentrated within the letter forms. So, abstractly the cells are defined by the letters and the surrounding images themselves. The “Tech Connect” sign was designed by Michael Thompson of Coded Meaning, a frequent collaborator with Ms. Wagner.'
print(properties_df.description.iloc[251])

Source material for the photos includes trees, rocks and plants from Overfelt Gardens, as well as Chinese Scholars’ Rocks from some of the preeminent collections in the United States.Integrated into the installation at the west end of the building is the signage for the Library’s “Tech Connect” area. This sign is comprised of two parts—the practical wayfinding text, and the surrounding cellular image forms. The cellular pattern was made using Voronoi diagrams for points that are algorithmically distributed through the panel and concentrated within the letter forms. So, abstractly the cells are defined by the letters and the surrounding images themselves. The “Tech Connect” sign was designed by Michael Thompson of Coded Meaning, a frequent collaborator with Ms. Wagner
[]
Internationally-recognized photographer Catherine Wagner created a dynamic architectural frieze that investigates ideas about knowledge, nature, wonder and discovery. This frieze is installed on the clerestory walls of 

In [48]:
#convert properties_df back to json and load into a dict
properties_json = properties_df.to_json(orient='records')
properties_json_dict = json.loads(properties_json)

In [49]:
#now get the properties json dict for each row(or record) 
#and replace the properties column in the features df
#with updated properties

a=[]
for record in range(len(properties_json_dict)):
    a.append(properties_json_dict[record])

    
#dropping rows from features_df
features_df = features_df.drop([83,179,59,173,14,82,184,137,227, 264, 106,178])
features_df.reset_index(drop=True,inplace=True)

features_df['properties']=a
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 6 columns):
addToMap      1 non-null float64
errors        3 non-null object
geometry      258 non-null object
id            258 non-null int64
properties    258 non-null object
type          258 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 12.2+ KB


In [63]:
features_df[features_df.addToMap.isnull()==0]

,addToMap,errors,geometry,id,properties,type
123,0.0,[coordinates],"{'type': 'Point', 'coordinates': None}",1924483022,"{'address': 'McAbee Road & Meridian Avenue', 'artist': 'Gary Alsum', 'city': 'San Jose', 'country': None, 'crossStreet': None, 'description': 'This artwork was commissioned to commemorate Jeffrey Fontana, the young San Jose police officer killed in the line of duty. Mr. Fontana was a 24-year-old Woodside resident and rookie police officer patrolling the upscale Almaden Valley neighborhood when he was fatally shot during a routine stop.', 'image': 'http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000000000%5c1760%5c2418%5c2803%2fImages6.8.11+206_201405142003236762.jpg&thumbnailSize=Thumb400X400&preserveAspect=true', 'postalCode': 95120, 'sourceOfInformation': '', 'sourceURL': 'http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Jeffrey-Fontana-Memorial-Font-506', 'state': 'CA', 'title': 'Jeffrey Fontana Memorial', 'website': '['https://garyalsum.com/']', 'AdditionalURL': []}",Feature


In [64]:
features_df[features_df.errors.isnull()==0]

,addToMap,errors,geometry,id,properties,type
120,NaN,"[title is ""untitled"", check other details]","{'type': 'Point', 'coordinates': [-121.8194551, 37.2593775]}",1869839991,"{'address': '300 Edenvale Ave', 'artist': 'Anna Valentina Murch', 'city': 'San Jose', 'country': None, 'crossStreet': None, 'description': 'Anna Valentina Murch’s Garden Park is situated in Edenvale Park next to the historic Hayes Mansion. Through public meetings, Murch discovered that the community wanted an artwork that was inspired by the vast history of the site. They asked for the piece to incorporate elements of the indigenous people of the valley, the agricultural era, Frontier Village - the 1960s era amusement park located on the site, the Hayes family orchards, and the garden of Mary Hayes. They also desired an artwork that revolved around a theme of transition, both physically and symbolically, to represent the transformation of the park through the master plan and the restoration of wildlife. Lastly, they hoped that Murch’s artwork would be a place of contemplation and peace. Murch’s artwork features a stone circle in the middle of the garden covered in round holes, reminiscent of Native American pounding holes, which function as waterholes for wildlife. To the north of the stone circle is an oak grove, which also references indigenous people who were dependent upon acorns as a staple of their diet. Ornamental cherry trees are planted to the west and east of the circle in remembrance of the Hayes’s orchard and the valley’s agricultural past. A bridge leads away from the circle towards the mansion over a moat of wildflowers in honor of Frontier Village. Lavender is found on the far edge of the oak grove, creating a rich scent nostalgic of Mary Hayes’ healing garden. This peaceful artwork provides an area for quiet meditation on the transition of the seasons and the transformation of the park’s historic value. Students from neighboring elementary, middle and high schools participated in the creation of the artwork and, in the past, used it as a touchstone for a curriculum about the community.', 'image': 'http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000000000%5c1760%5c2418%5c2883%2fedenvale0001_201406111814128639.jpg&thumbnailSize=Thumb400X400&preserveAspect=true', 'postalCode': 95136, 'sourceOfInformation': '', 'sourceURL': 'http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Untitled-Edenvale-Garden-Park-502', 'state': 'CA', 'title': 'Untitled', 'website': '['https://en.wikipedia.org/wiki/Anna_Valentina_Murch']', 'AdditionalURL': []}",Feature
123,0.0,[coordinates],"{'type': 'Point', 'coordinates': None}",1924483022,"{'address': 'McAbee Road & Meridian Avenue', 'artist': 'Gary Alsum', 'city': 'San Jose', 'country': None, 'crossStreet': None, 'description': 'This artwork was commissioned to commemorate Jeffrey Fontana, the young San Jose police officer killed in the line of duty. Mr. Fontana was a 24-year-old Woodside resident and rookie police officer patrolling the upscale Almaden Valley neighborhood when he was fatally shot during a routine stop.', 'image': 'http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000000000%5c1760%5c2418%5c2803%2fImages6.8.11+206_201405142003236762.jpg&thumbnailSize=Thumb400X400&preserveAspect=true', 'postalCode': 95120, 'sourceOfInformation': '', 'sourceURL': 'http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Jeffrey-Fontana-Memorial-Font-506', 'state': 'CA', 'title': 'Jeffrey Fontana Memorial', 'website': '['https://garyalsum.com/']', 'AdditionalURL': []}",Feature
194,NaN,"[title is ""untitled"", check other details]","{'type': 'Point', 'coordinates': [-121.858973, 37.3511716]}",3193533850,"{'address': '2949 Alum Rock Avenue', 'artist': 'Roberto Delgado', 'city': 'San Jose', 'country': None, 'crossStreet': None, 'description': 'Alarge exterior tile mural contains multiple layers of imagery from firefighting. 

In [57]:
# now write output to a file
jsDataFile = open('features.json', 'w') 
# magic happens here to make it pretty-printed
jsDataFile.write(json.dumps(json.loads(features_df.to_json(orient='records')), indent=4, sort_keys=True,ensure_ascii=Falsety))
jsDataFile.close()

In [53]:

#convert features df into json
features_json = features_df.to_json(orient='records')

#load features json into a dict
features_json_dict = json.loads(features_json)

#write features json dict to a json file in pretty format for QA and for later use
with open('features.json', 'w') as f:
    pprint.pprint(features_json_dict, stream=f)
f.close()
#pprint.pprint(features_json_dict)

# don't do this b/c the resultant df doesn't give back original json format

#now get the features json dict for each row(or record) 
#and replace the features column in df
#with updated features

a=[]
for record in range(len(features_json_dict)):
    a.append(features_json_dict[record])
    
df['features']=a
df.head()

# Load original json into a dict and replace features data with features_json_dict

In [59]:
with open(fname) as f:
    original_dict = json.load(f)
f.close()

original_dict['features'] = features_json_dict

with open('consolidated-artworks-fixed.json', 'w') as f:
    pprint.pprint(original_dict, stream=f)
f.close()

#used original_dict.keys()

In [64]:
pprint.pprint(original_dict)

{'features': [{'addToMap': None,
               'errors': None,
               'geometry': {'coordinates': [-121.88887, 37.32718],
                            'type': 'Point'},
               'id': 10667523,
               'properties': {'address': '488 Almaden Blvd, Ste 1800',
                              'artist': 'J.Duh',
                              'artist_site': 'https://www.instagram.com/j.duh_/',
                              'city': 'San José',
                              'country': 'United States',
                              'crossStreet': None,
                              'description': 'This mural was painted by <a '
                                             "href='https://www.instagram.com/j.duh_/' "
                                             "target='_blank'>J.Duh</a>, and "
                                             'produced in collaboration with '
                                             '<a '
                                             "href='http

                                             'changes based on environment. '
                                             'The public will observe this '
                                             'relationship between scale and '
                                             'environment when standing next '
                                             'to these grand seven foot tall '
                                             'sculptures.',
                              'image': 'http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000000000%5c1760%5c2418%5c2742%2fKaneko_63_201404301344087749.jpg&thumbnailSize=Thumb400X400&preserveAspect=true',
                              'postalCode': 95113,
                              'sourceOfInformation': 'http://sanjoseca.gov/Facilities',
                              'sourceURL': 'http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Convergence-San-Jose-Repertor-418',
                     

                                             'was often said that she could '
                                             '“teach a rock to read.” \xa0'
                                             'Teaching Stones consists of two '
                                             'primary elements. The first is a '
                                             'reading/sitting area, inspired '
                                             'by one of Doris’ favorite '
                                             'activities: story telling. The '
                                             'reading area provides a place to '
                                             'sit and share stories in the '
                                             'traditional manner of gathering '
                                             'in a circle. The decomposed '
                                             'granite interior of the circle '
                                             'extends outwar

               'properties': {'address': '2031 Pulgas Avenue',
                              'artist': 'MMAP (Mural Music and Arts Project)',
                              'artist_site': 'http://www.muralmusicarts.org/',
                              'city': 'East Palo Alto',
                              'country': 'United States',
                              'crossStreet': None,
                              'description': 'In 2001, <a '
                                             "href='http://www.muralmusicarts.org/' "
                                             "target='_blank'>MMAP (Mural "
                                             'Music and Arts Project)</a> and '
                                             'lead artist Omar Ramirez '
                                             'collaborated with students and '
                                             'staff at Boys & Girls Clubs of '
                                             'the Peninsula - Moldaw-Zaffaroni '


                                             'dances.',
                              'image': 'http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000000000%5c1760%5c2418%5c2878%2fMvc-002l_201406111803009248.jpg&thumbnailSize=Thumb400X400&preserveAspect=true',
                              'postalCode': 95112,
                              'sourceOfInformation': 'http://sanjoseca.gov/Facilities',
                              'sourceURL': 'http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Transcendance-Northside-Commu-453',
                              'state': 'CA',
                              'title': 'Transcendance',
                              'website': None},
               'type': 'Feature'},
              {'addToMap': None,
               'errors': None,
               'geometry': {'coordinates': [-121.86185, 37.33739],
                            'type': 'Point'},
               'id': 688518587,
               'propertie

...

                                             'months with the City of San '
                                             "Jose's Cultural Affairs Office "
                                             'and the Environmental Services '
                                             "Department. He is the City's "
                                             'first Photographer-in-Residence '
                                             'at their Wastewater Treatment '
                                             'Facility. Like ancient Rome our '
                                             'contemporary culture was made '
                                             'possible by a vast system of '
                                             'water delivery and sewage '
                                             'treatment to maintain a large '
                                             'and healthy population. Most of '
                                             'us are generally u

                                             'figure skating champions: Brian '
                                             'Boitano,\xa0Peggy '
                                             'Fleming-Jenkins, Rudy Galindo, '
                                             'Debi Thomas and\xa0Kristi '
                                             'Yamaguchi. Five Skaters honors '
                                             'the art and sport of\xa0figure '
                                             'skating, San Jose’s local '
                                             'champions and others who\xa0have '
                                             'contributed to its vitality, '
                                             'tradition and popularity.\xa0The '
                                             'artwork consists of three parts: '
                                             'five monumental\xa0columns, a '
                                             'large terrazzo ice rink, 

                            'type': 'Point'},
               'id': 3693605911,
               'properties': {'address': '45 N 1st Street',
                              'artist': 'Ricky Watts',
                              'artist_site': '',
                              'city': 'San José',
                              'country': 'United States',
                              'crossStreet': None,
                              'description': 'Mural exhibited at the 2017 <a '
                                             "href='https://www.powwowsanjose.com/' "
                                             "target='_blank'>POW! WOW! San "
                                             'José</a> in collaboration with '
                                             '<a '
                                             "href='https://www.empiresevenstudios.com' "
                                             "target='_blank'>Empire Seven "
                                             'Studios</a>.

               'errors': None,
               'geometry': {'coordinates': [-121.891267, 37.3471033],
                            'type': 'Point'},
               'id': 3971229800,
               'properties': {'address': '488 North 6th',
                              'artist': 'Peter Molica',
                              'artist_site': '',
                              'city': 'San Jose',
                              'country': None,
                              'crossStreet': None,
                              'description': 'Capiz, a series of stained-glass '
                                             'artisan windows welcome all '
                                             'visitors entering the main foyer '
                                             'of the JTC Northside Community '
                                             'Center in San Jose.\xa0 '
                                             'Kaleidoscopic and delicate, the '
                                        

                              'sourceOfInformation': None,
                              'sourceURL': None,
                              'state': 'CA',
                              'title': 'La Gran Cultura Resonance',
                              'website': None},
               'type': 'Feature'},
              {'addToMap': None,
               'errors': None,
               'geometry': {'coordinates': [-121.88019, 37.32226],
                            'type': 'Point'},
               'id': 4153382864,
               'properties': {'address': '921 S 1st St',
                              'artist': 'Maria Alguilar',
                              'artist_site': '',
                              'city': 'San José',
                              'country': 'United States',
                              'crossStreet': None,
                              'description': 'The artwork was created in 1991. '
                                             'The work is a story of two '
       

# Flatten json to fit in pandas dataframe

In [ ]:
import json 
import pandas as pd 
from pandas.io.json import json_normalize #package for flattening json in pandas df

#load json object
with open(fname) as f:
    d = json.load(f)

#lets put the data into a pandas df
#clicking on consolidated-artworks.json under "Input Files"
#tells us parent node WE WANT TO EDIT is 'features'
artwrk = json_normalize(d['features'])
artwrk.head(len(artwrk))

# edit 'properties.artist' column keeping text tag as name and putting attrs into 'properties.artist_site'

In [ ]:
artwrk_cpy.head()

# title column looks clean

In [ ]:
artwrk['properties.title']

In [ ]:
artwrk['properties.title'].iloc[41]